In [1]:
import uuid
from math import ceil
from time import sleep

In [2]:
import requests
import weaviate
from sentence_transformers import SentenceTransformer
from weaviate.classes.query import MetadataQuery, Metrics, QueryReference, Sort
from weaviate.collections.classes.filters import Filter

/workspaces/data-playground/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
def print_result_head(obj_list, length=3, max_truncation=75):
    for obj in obj_list[0:length]:
        print(
            obj.properties["category"].strip(),
            "-",
            (
                obj.properties["description"].strip()
                if len(obj.properties["description"].strip()) <= max_truncation
                else obj.properties["description"].strip()[0:max_truncation].strip()
                + " [...]"
            ),
        )

In [4]:
def print_result_distinct_head(obj_list, length=3, max_truncation=75):
    obj_set = sorted(
        {
            (
                o.properties["category"].strip().casefold()
                + " - "
                + (
                    o.properties["description"].strip().casefold()
                    if len(o.properties["description"].strip()) <= max_truncation
                    else o.properties["description"]
                    .strip()[0:max_truncation]
                    .strip()
                    .casefold()
                    + " [...]"
                )
            )
            for o in obj_list[0:length]
        },
    )
    print(*obj_set, sep="\n")

In [5]:
def print_result_distinct_classified_head(obj_list, length=3, max_truncation=75):
    obj_set = sorted(
        {
            (
                o.properties["category"].strip().casefold()
                + " - "
                + o.references["classifiedCategory"]
                .objects[0]
                .properties["name"]
                .strip()
                .casefold()
                + " - "
                + (
                    o.properties["description"].strip().casefold()
                    if len(o.properties["description"].strip()) <= max_truncation
                    else o.properties["description"]
                    .strip()[0:max_truncation]
                    .strip()
                    .casefold()
                    + " [...]"
                )
            )
            for o in obj_list[0:length]
        },
    )
    print(*obj_set, sep="\n")

In [6]:
HOST = "weaviate"
PORT = 8080
URL = f"http://{HOST}:{PORT}"
COLLECTION = "DataPlayground_Products"
CAT_COLLECTION = "DataPlayground_Categories"

In [7]:
client = weaviate.connect_to_local(host=HOST, port=PORT)
collection = client.collections.get(COLLECTION)

In [8]:
client.is_connected()

True

In [9]:
client.is_ready()

True

In [10]:
client.is_live()

True

In [11]:
client.get_meta()["version"]

'1.26.1'

In [12]:
client.get_meta()["hostname"]

'http://[::]:8080'

In [13]:
client.get_meta()["modules"].keys()

dict_keys(['backup-filesystem', 'qna-transformers', 'text2vec-transformers'])

In [14]:
collection.aggregate.over_all(total_count=True)

AggregateReturn(properties={}, total_count=50425)

In [15]:
res = collection.query.fetch_object_by_id("0003a929-338f-4174-84de-91963f0b97a7")
print_result_distinct_head([res])

household - nurturing green lucky bamboo two layer round glass pot two layered bamboo i [...]


In [16]:
res = collection.query.fetch_objects(
    filters=Filter.by_property("category").equal("Books"),
).objects
print_result_distinct_head(res)

books - inquilab: bhagat singh on religion & revolution about the author s irfan ha [...]
books - milk and honey review "at age 24, rupi kaur has been called the voice of he [...]
books - monk who sold his ferrari from the back cover wisdom to create a life of pa [...]


In [17]:
res = collection.query.near_text(["math"], certainty=0.75).objects
print_result_distinct_head(res, -1)

books - cbse ncert solutions mathematics 12  for 2018 - 19
books - crash course jee(main)/aieee - mathematics: working tricks to score high in [...]
books - engineering mathematics for gate & ese (prelims) 2019 - theory & previous y [...]
books - gmat math prep course
books - handbook of physics, chemistry, math
books - higher engineering mathematics
books - mathematics
books - mathematics for jee main
books - mathematics for joint entrance examination jee (advanced) algebra
books - mathematics for joint entrance examination jee (advanced) coordinate geomet [...]
books - mathematics formulae for school students: clear your basic concepts to solv [...]
books - space mathematics: math problems based on space science


In [18]:
res = collection.query.near_text(["learning"], certainty=0.75).objects
print_result_distinct_head(res, -1)

books - becoming equipped to communicate: a practical guide for learning a language [...]
books - case studies mentor in business studies class - xii (2018-19 session)
books - creative writing: a beginner’s manual, 1e
books - digital photography complete course
books - express learning - artificial intelligence, 1e
books - general training student's book with answers with audio
books - group discussion - more than 100 topics covered about the author the author [...]
books - how to train in archery: being a complete study of the york round
books - looking around - textbook in environmental studies for class - 3  - 327
books - looking around - textbook in environmental studies for class - 4  - 427
books - second language teaching and learning with technology: views of emergent re [...]
books - the business school
books - unlimited memory: how to use advanced learning strategies to learn faster, [...]
books - young learners, environmental studies, book 2
electronics - laptop repairing cou

In [19]:
res = collection.query.near_text(["learn mathematics"], certainty=0.75).objects
print_result_distinct_head(res, -1)

books - 24 sample question papers for cbse class 12 physics, chemistry, mathematics [...]
books - 35 jee main online & offline physics, chemistry & mathematics topic-wise so [...]
books - crash course jee(main)/aieee - mathematics: working tricks to score high in [...]
books - gmat math prep course
books - handbook of physics, chemistry, math
books - higher engineering mathematics
books - mathematics
books - mathematics for joint entrance examination jee (advanced) coordinate geomet [...]
books - mathematics formulae for school students: clear your basic concepts to solv [...]
books - olympiad champs mathematics class 1 with past olympiad questions
books - olympiad champs mathematics class 3 with past olympiad questions
books - olympiad champs mathematics class 7 with past olympiad questions
books - space mathematics: math problems based on space science
books - vedic mathematics made easy vedic mathematics made easy


In [20]:
res = collection.query.near_text(["learn", "mathematics"], certainty=0.75).objects
print_result_distinct_head(res, -1)

books - case studies mentor in business studies class - xii (2018-19 session)
books - cbse ncert solutions mathematics 12  for 2018 - 19
books - engineering mathematics for gate & ese (prelims) 2019 - theory & previous y [...]
books - higher engineering mathematics
books - international mathematics olympiad work book (imo) - class 3 for 2018-19
books - mastering the market cycle: getting the odds on your side
books - mathematics
books - mathematics for jee main
books - olympiad champs mathematics class 3 with past olympiad questions
books - the quick and easy way to effective speaking
books - the way to geometry (perfect library)


In [21]:
res = collection.query.near_text(["computer science"], certainty=0.75).objects
print_result_distinct_head(res, -1)

books - a handbook on computer science & it - illustrated formulae & key theory con [...]
books - computer fundamentals, 1e
books - computer networking: a top-down approach
books - gate 2019: computer science and it engineering - previous solved papers
books - gate computer science & information technology (27 year’s chapter wise solv [...]
books - gate guide computer science and information technology 2019
books - introduction to computer security, 1e
books - isro 2019: computer science - previous years solved papers
books - science and technology for upsc and state civil services examinations about [...]
books - ugc net/jrf exam solved papers computer science & applications it is a book
books - ugc-net 2019: computer science & applications (paper ii & paper iii)
books - what algorithms want: imagination in the age of computing


In [22]:
res = collection.query.near_text(["python"], certainty=0.75).objects
print_result_distinct_head(res, -1)

books - algorithms
books - beautiful javascript beautiful javascript leading programmers explain how t [...]
books - computer fundamentals, 1e
books - curriculum [...]
books - data structure and algorithmic thinking with python
books - express learning - artificial intelligence, 1e
books - gate 2019: computer science and it engineering - previous solved papers
books - introduction to computer security, 1e
books - mathematics
books - python programming: a modular approach by pearson
books - python: the complete reference
books - the way to geometry (perfect library)


In [23]:
res = collection.query.near_text(["learn", "python"], certainty=0.75).objects
print_result_distinct_head(res, -1)

books - becoming
books - becoming equipped to communicate: a practical guide for learning a language [...]
books - case studies mentor in business studies class - xii (2018-19 session)
books - computer fundamentals, 1e
books - creative writing: a beginner’s manual, 1e
books - express learning - artificial intelligence, 1e
books - introduction to computer security, 1e
books - mastering the market cycle: getting the odds on your side
books - no point talking.
books - python programming: a modular approach by pearson
books - python: the complete reference
books - thank you for arguing
books - the way to geometry (perfect library)


In [24]:
res = collection.query.near_text(["ww2"], certainty=0.75).objects
print_result_distinct_head(res, -1)

books - a brief history of time: from big bang to black holes
books - international relations and politics: theoretical perspectives in the post- [...]
books - international relations: perspectives for the global south, 1e
books - our pasts 3 part - 2 textbook in history for class 8- 862
books - rise and fall of the third reich-30th anniv edition: a history of nazi germ [...]
books - the art of war
books - the art of war: the oldest military treatise in the world
books - the military history book: the ultimate visual guide to the weapons that sh [...]


In [25]:
res = collection.query.near_text(["gardening"], certainty=0.75).objects
print_result_distinct_head(res, -1)

books - a concise guide to roses
books - cell biology, genetics, molecular biology, evolution & ecology
books - fundamentals of plant physiology
books - gardening in urban india update
books - organic home gardening made easy “organic home gardening made easy” opens u [...]
books - plant anatomy
books - the little book of gardening tips
books - the wisdom of sundays
books - think and grow rich
household - saingace 1 pcs artificial flower vine hanging garland plant for home living [...]


In [26]:
model = SentenceTransformer("Snowflake/snowflake-arctic-embed-xs")
sentence = "Gardening guide"
embedding_vector = model.encode(sentence, normalize_embeddings=True)
print("Embedding shape: ", embedding_vector.shape)
print("Embedding head: ", embedding_vector[0:5])

res = collection.query.near_vector(embedding_vector, certainty=0.75).objects
print("Search results:")
print_result_distinct_head(res, -1)

Embedding shape:  (384,)
Embedding head:  [ 0.03122007  0.04399511  0.04697644 -0.0308421  -0.03561646]
Search results:
books - a concise guide to roses
books - cell biology, genetics, molecular biology, evolution & ecology
books - encyclopedia golf techniques
books - food nutrition and diet therapy agricultural guide
books - fundamentals of plant physiology
books - gardening in urban india update
books - on the origin of species
books - plant anatomy
books - the little book of gardening tips
books - think and grow rich


In [27]:
model = SentenceTransformer("Snowflake/snowflake-arctic-embed-xs")
sentence = "Greatest literature pieces everyone should read"
embedding_vector = model.encode(sentence, normalize_embeddings=True)
print("Embedding shape: ", embedding_vector.shape)
print("Embedding head: ", embedding_vector[0:5])

res = collection.query.near_vector(embedding_vector, certainty=0.75).objects
print("Search results:")
print_result_distinct_head(res, -1)

Embedding shape:  (384,)
Embedding head:  [ 0.04768769  0.04728265  0.0009177  -0.06762231 -0.02431852]
Search results:
books - an outline history of english literature
books - english literature: modern
books - essays in literature & history
books - history of english literature
books - history of english literature (old edition) good books for reading.
books - modern literary theory: a comparative introduction


In [28]:
model = SentenceTransformer("Snowflake/snowflake-arctic-embed-xs")
sentence = "Ocean level rising due to climate change"
embedding_vector = model.encode(sentence, normalize_embeddings=True)
print("Embedding shape: ", embedding_vector.shape)
print("Embedding head: ", embedding_vector[0:5])

res = collection.query.near_vector(embedding_vector, certainty=0.75).objects
print("Search results:")
print_result_distinct_head(res, -1)

Embedding shape:  (384,)
Embedding head:  [ 0.00537249  0.05073355  0.00906473  0.00589836 -0.06142267]
Search results:
books - csir-ugc net/jrf earth, atmospheric, ocean and planetary sciences exam guid [...]
books - earth: an introduction to physical geology, global edition
books - environmental engineering science
books - environmental studies, 1e
books - into thin air
books - into thin air update
books - introduction to enviromental engineering
books - let's explore... ocean (lonely planet kids)
books - looking around - textbook in environmental studies for class - 3  - 327
books - looking around - textbook in environmental studies for class - 4  - 427
books - love with a chance of drowning
books - mastering the market cycle: getting the odds on your side
books - sail
books - sailing alone around the world
books - the complete guide to airgunning
books - the earth


In [29]:
model = SentenceTransformer("Snowflake/snowflake-arctic-embed-xs")
sentence = (
    "A timeless tragedy that explores the themes of love, fate, and "
    "the struggle between individual desires and societal constraints."
)
embedding_vector = model.encode(sentence, normalize_embeddings=True)
print("Embedding shape: ", embedding_vector.shape)
print("Embedding head: ", embedding_vector[0:5])

res = collection.query.near_vector(embedding_vector, certainty=0.75).objects
print("Search results:")
print_result_distinct_head(res, -1)

Embedding shape:  (384,)
Embedding head:  [ 0.03968731  0.0350755   0.01525651 -0.02734353 -0.04829176]
Search results:
books - 100 world’s greatest love poems
books - a vindication of the rights of woman (amazonclassics edition)
books - a vindication of the rights of woman: with strictures on political and mora [...]
books - an introduction to philosophy
books - conscious loving: the journey to co-commitment
books - english literary criticism and theory
books - love with a chance of drowning
books - love with a chance of drowning: a memoir
books - metaphysics translation commentary and introduction
books - modern literary theory: a comparative introduction
books - political theory: an introduction
books - relativity: the special and general theory
books - speak to win: how to present with power in any situation
books - the better angels of our nature: why violence has declined
books - the consolation of philosophy
books - the strange death of europe: immigration, identity, islam
books

In [30]:
model = SentenceTransformer("Snowflake/snowflake-arctic-embed-xs")
sentence = "romeo and juliet"
embedding_vector = model.encode(sentence, normalize_embeddings=True)
print("Embedding shape: ", embedding_vector.shape)
print("Embedding head: ", embedding_vector[0:5])

res = collection.query.near_vector(embedding_vector, certainty=0.75).objects
print("Search results:")
print_result_distinct_head(res, -1)

Embedding shape:  (384,)
Embedding head:  [ 0.04227388  0.00320177  0.01457122 -0.04516331 -0.0235528 ]
Search results:
books - english language and linguistics: a simplified approach
books - english literary criticism and theory
books - english literature: modern
books - history of english literature
books - hunting with the bow and arrow
books - modern literary theory: a comparative introduction
books - romeo and juliet
books - romeo and juliet (graphic shakespeare audio edition) about the author willi [...]
books - romeo and juliet (shakespeare, signet classic) review "we can more easily d [...]
books - romeo and juliet about the author william shakespeare, perhaps the greatest [...]
books - romeo and juliet audio cassette set (3 cassettes) (new cambridge shakespear [...]


In [31]:
for _ in range(
    ceil(collection.aggregate.over_all(total_count=True).total_count / 10000),
):
    weaviate.Client(URL).classification.schedule().with_type(
        "zeroshot",
    ).with_class_name(
        COLLECTION,
    ).with_based_on_properties(
        ["description"],
    ).with_classify_properties(
        ["classifiedCategory"],
    ).with_wait_for_completion().do()

/workspaces/data-playground/.venv/lib/python3.12/site-packages/weaviate/warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


In [32]:
res = collection.query.fetch_objects(
    return_references=[
        QueryReference(
            link_on="classifiedCategory",
            return_properties=["name"],
        ),
    ],
    limit=3,
).objects
for o in res:
    original_cat = o.properties["category"]
    classified_cat = o.references["classifiedCategory"].objects[0].properties["name"]
    if original_cat != classified_cat:
        print("Classification MISSED!")
    else:
        print("Classification HIT!")
    print(
        f"Original category: {original_cat}",
        f"Classified category: {classified_cat}",
        "Description: "
        + (
            o.properties["description"].strip()
            if len(o.properties["description"].strip()) <= 750
            else o.properties["description"].strip()[0:750].strip() + " [...]"
        ),
        sep="\n",
    )
    print()

Classification HIT!
Original category: Books
Classified category: Books
Description: Grandma Gatewood's Walk: The Inspiring Story of the Woman Who Saved the Appalachian Trail Review "Grandma Gatewood's Walk is sure to fuel not only the dreams of would-be hikers, but debates on the limits of endurance, the power of determination, and the nature of myth." --EARL SWIFT, AUTHOR OF THE BIG ROADS"Go, Granny, Go! . . . This astonishing tale will send you looking for your hiking boots. A wonderful story, wonderfully told." --CHARLES MCNAIR, BOOKS EDITOR AT PASTE MAGAZINE AND AUTHOR OF PICKETT'S CHARGE"A quiet delight of a book." --KIRKUS REVIEWS"Grandma Gatewood's Walk is a brilliant look at an America--both good and bad--that has slipped away, seen through the eyes and feet of one of America'smost unlikely heroines. Gatewood's sto [...]

Classification HIT!
Original category: Clothing & Accessories
Classified category: Clothing & Accessories
Description: Criba Anti-Reflective Rectangular Unis

In [33]:
res = collection.iterator(
    return_references=[
        QueryReference(
            link_on="classifiedCategory",
            return_properties=["name"],
        ),
    ],
)
out = {"success": 0, "failure": 0}
for o in res:
    original_cat = o.properties["category"]
    classified_cat = o.references["classifiedCategory"].objects[0].properties["name"]
    if original_cat != classified_cat:
        out["failure"] = out["failure"] + 1
    else:
        out["success"] = out["success"] + 1
success = out["success"]
failure = out["failure"]
success_percentage = success / (success + failure)
print(
    f"Hit: {success}",
    f"Miss: {failure}",
    f"Hit score: {success_percentage}",
    sep="\n",
)

Hit: 29332
Miss: 21093
Hit score: 0.5816955875061973


In [34]:
res = client.graphql_raw_query(
    """
  {
    Get {
      DataPlayground_Products(
        ask: {
          question: "How many books are there in 'My Complete Kit of Nursery Books'?",
          properties: ["description"],
        },
        limit: 5
      ) {
        description
        _additional {
          answer {
            certainty
            hasAnswer
            property
            result
            startPosition
            endPosition
          }
        }
      }
    }
  }
  """,
)
[
    (
        answer["_additional"]["answer"]["result"],
        answer["_additional"]["answer"]["certainty"],
        answer["description"],
    )
    for answer in res.get["DataPlayground_Products"]
    if answer["_additional"]["answer"]["hasAnswer"]
]

[('9',
  0.5212255477905273,
  'My Complete Kit of Nursery Books - A Set of 9 Books '),
 ('9',
  0.5212255477905273,
  'My Complete Kit of Nursery Books - A Set of 9 Books '),
 ('8',
  0.5125787734985352,
  'My Complete Kit of Pre-Nursery Books - A Set of 8 Books '),
 ('10',
  0.41549291610717776,
  'Lovely Board Books: Gift Pack (Set of 10 Books) '),
 ('10',
  0.41549291610717776,
  'Lovely Board Books: Gift Pack (Set of 10 Books) ')]

In [35]:
res = client.graphql_raw_query(
    """
  {
    Get {
      DataPlayground_Products(
        ask: {
          question: "What is the size of 'Rasper Clear Acrylic Table Top Elevator'?",
          properties: ["description"],
        },
        limit: 5
      ) {
        description
        _additional {
          answer {
            certainty
            hasAnswer
            property
            result
            startPosition
            endPosition
          }
        }
      }
    }
  }
  """,
)
[
    (
        answer["_additional"]["answer"]["result"],
        answer["_additional"]["answer"]["certainty"],
        answer["description"],
    )
    for answer in res.get["DataPlayground_Products"]
    if answer["_additional"]["answer"]["hasAnswer"]
]

[('small / 16x12 inches',
  0.7831254482269288,
  'Rasper Acrylic Table Top Elevator Writing Desk with Adjustable Height (Small/16x12 Inches) A table top elevator writing desk with adjustable height. Rasper brings you the unique, elegant and widest range of acrylic products which includes office stationery and supplies, corporate, and personal products.'),
 ('16 * 12 inches',
  0.7955202102661133,
  'Rasper Clear Acrylic Table Top Elevator Writing Desk (Small Size 16*12 Inches) With Adjustable Height'),
 ('16 * 12 inches',
  0.8252475738525391,
  'Rasper Acrylic Table Top Elevator Writing Desk (Small Size 16*12 Inches) With Adjustable Height Rasper presents you the unique, elegant and widest range of acrylic products which includes office stationery and supplies, corporate, promotional and personal Gifting, the brand comes from the house of Shivam Acrylic Products.'),
 ('11 \', 9 ", 7 "',
  0.18073320388793945,
  'Kitchenexp :Unbreakable" Premium Quality Crystal Clear Corner Set Plasti

In [36]:
res = client.graphql_raw_query(
    """
  {
    Get {
      DataPlayground_Products(
        ask: {
          question: "What is the school from which Anurag K. Agarwal graduated?",
          properties: ["description"],
        },
        limit: 5
      ) {
        description
        _additional {
          answer {
            certainty
            hasAnswer
            property
            result
            startPosition
            endPosition
          }
        }
      }
    }
  }
  """,
)
[
    (
        answer["_additional"]["answer"]["result"],
        answer["_additional"]["answer"]["certainty"],
        answer["description"],
    )
    for answer in res.get["DataPlayground_Products"]
    if answer["_additional"]["answer"]["hasAnswer"]
]

[('harvard law school',
  0.8910634994506836,
  "Business Law for Managers: IIMA Series About the Author Having graduated from Harvard Law School, Anurag K. Agarwal was the first recipient of the Marti Mannariah Gurunath Outstanding Teacher Award at Indian Institute of Management, Ahmedabad. His teaching, consulting and research interests include business and intellectual property, business leadership, contracts and arbitration, infrastructure and PPPs, medico-legal issues in healthcare and related issues. Agarwal conducts executive education for the government as well as public and private sector companies. He was a practicing advocate in Lucknow for seven years before switching to teaching. Agarwal writes a weekly column 'Lawfully Yours' for DNA, Ahmedabad. He has authored three books-Business Leadership and Law (2017), Contracts and Arbitration for Managers (2016) and Business and Intellectual Property (2010)."),
 ('harvard law school',
  0.8910634994506836,
  "Business Law for Mana

In [37]:
res = collection.query.fetch_objects(
    filters=Filter.by_property("category").equal("Books"),
    sort=Sort.by_property("description"),
).objects
print_result_distinct_head(res, 10)

books - #horror
books - 1 childhood & growing up2contemporary india and education paper 3 learning [...]
books - 1,000 places to see before you die (1,000... before you die books) review " [...]
books - 1,000 places to see before you die review "at last, a book that tells you w [...]
books - 1,001 gre practice questions for dummies: with free online practice


In [38]:
res = collection.query.fetch_objects(
    filters=Filter.by_property("category").equal("Books"),
    sort=Sort.by_property("description"),
    limit=10,
).objects
print_result_head(res, -1)

Books - #Horror
Books - #Horror
Books - #Horror
Books - #Horror
Books - 1 Childhood & growing up2Contemporary india and education PAPER 3 LEARNING [...]
Books - 1 Childhood & growing up2Contemporary india and education PAPER 3 LEARNING [...]
Books - 1,000 Places to See Before You Die (1,000... Before You Die Books) Review " [...]
Books - 1,000 Places to See Before You Die Review "At last, a book that tells you w [...]
Books - 1,000 Places to See Before You Die Review "At last, a book that tells you w [...]


In [39]:
res = collection.query.fetch_objects(
    filters=Filter.by_property("category").equal("Books"),
    sort=Sort.by_property("description"),
    limit=10,
    offset=4,
).objects
print_result_head(res, -1)

Books - 1 Childhood & growing up2Contemporary india and education PAPER 3 LEARNING [...]
Books - 1 Childhood & growing up2Contemporary india and education PAPER 3 LEARNING [...]
Books - 1,000 Places to See Before You Die (1,000... Before You Die Books) Review " [...]
Books - 1,000 Places to See Before You Die Review "At last, a book that tells you w [...]
Books - 1,000 Places to See Before You Die Review "At last, a book that tells you w [...]
Books - 1,001 GRE Practice Questions for Dummies: With Free Online Practice
Books - 1,001 Tips For Orthodontics And Its Secrets About the Author Dr Esequiel E [...]
Books - 10 Actual, Official LSAT PrepTests (LSAT Series)
Books - 10 Actual, Official LSAT PrepTests (LSAT Series)


In [40]:
res = collection.aggregate.over_all(
    return_metrics=Metrics("category").text(
        top_occurrences_count=True,
        top_occurrences_value=True,
    ),
)
res.properties["category"].top_occurrences

[TopOccurrence(count=19313, value='Household'),
 TopOccurrence(count=11820, value='Books'),
 TopOccurrence(count=10621, value='Electronics'),
 TopOccurrence(count=8671, value='Clothing \\u0026 Accessories')]

In [41]:
res = collection.aggregate.near_text(
    query="flowers",
    certainty=0.85,
    total_count=True,
    return_metrics=Metrics("category").text(
        top_occurrences_count=True,
        top_occurrences_value=True,
    ),
)
res.properties["category"].top_occurrences

[TopOccurrence(count=9, value='Household'),
 TopOccurrence(count=2, value='Books'),
 TopOccurrence(count=2, value='Clothing & Accessories')]

In [42]:
res = collection.query.near_text(
    ["astronomy"],
    return_metadata=MetadataQuery(distance=True),
).objects
[r.metadata.distance for r in res][0:3]

[0.19401127099990845, 0.20127767324447632, 0.20342302322387695]

In [43]:
res = collection.query.near_text(
    ["astronomy"],
    return_metadata=MetadataQuery(distance=True),
    distance=0.2,
).objects
[r.metadata.distance for r in res][0:3]

[0.19401127099990845]

In [44]:
backup_name = str(uuid.uuid4())

In [45]:
response = requests.post(
    f"{URL}/v1/backups/filesystem",
    headers={"Content-Type": "application/json"},
    data=f'{{"id": "{backup_name}"}}',
    timeout=30,
)
response.json()

{'backend': 'filesystem',
 'classes': ['DataPlayground_Categories', 'DataPlayground_Products'],
 'id': '10be03d9-fa9b-46ad-b999-c3739556078e',
 'path': '/tmp/backups/10be03d9-fa9b-46ad-b999-c3739556078e',
 'status': 'STARTED'}

In [46]:
sleep(60)
response = requests.get(
    f"{URL}/v1/backups/filesystem/{backup_name}",
    headers={"Content-Type": "application/json"},
    timeout=30,
)
response.json()

{'backend': 'filesystem',
 'id': '10be03d9-fa9b-46ad-b999-c3739556078e',
 'path': '/tmp/backups/10be03d9-fa9b-46ad-b999-c3739556078e',
 'status': 'SUCCESS'}

In [47]:
response = requests.post(
    f"{URL}/v1/backups/filesystem/{backup_name}/restore",
    headers={"Content-Type": "application/json"},
    data=f'{{"id": "{backup_name}"}}',
    timeout=30,
)
response.json()

{'backend': 'filesystem',
 'classes': ['DataPlayground_Products', 'DataPlayground_Categories'],
 'id': '10be03d9-fa9b-46ad-b999-c3739556078e',
 'path': '/tmp/backups/10be03d9-fa9b-46ad-b999-c3739556078e',
 'status': 'STARTED'}